![banner](https://bccfilkom.net/static/assets/images/BCC-Logo.svg)

# Mini Bootcamp Data Science Day-1

### **TO BE A DATA SCIENTIST**

## Intro Machine Learning

### Apa itu ML?
metode untuk menyelesaikan masalah tanpa harus menjabarkan aturan secara eksplisit. Machine learning dapat belajar dan beradaptasi melalui data yang diberikan. 

![wiml](https://i.imgur.com/guZJACw.png)

### Tipe Machine Learning

Berdasarkan Cara Belajar

1. Supervised -> Diberi contoh yang bener kayak gimana
2. Unsupervised -> Belajar sendiri
3. dll.. 

Berdasarkan task-nya
1. Regresi -> saham, curah hujan, harga bawang
2. Klasifikasi -> tipe, kelas

## Review Day 1

In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

d:\Python\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Python\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Python\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/SulthanAbiyyu/mini-bootcamp-bcc-2023/master/Supermarket_Sales.csv",
)
data.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7.0,26.1415,548.9715,1/5/2019,13:08,Ewallet,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5.0,3.8200,80.2200,3/8/2019,10:29,Cash,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7.0,16.2155,340.5255,3/3/2019,13:23,Credit card,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8.0,23.2880,489.0480,1/27/2019,20:33,Ewallet,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7.0,30.2085,634.3785,2/8/2019,10:37,Ewallet,5.3


In [4]:
print("Banyak baris duplikat: ", data.duplicated().sum())

Banyak baris duplikat:  49


In [5]:
data = data.drop_duplicates()

In [6]:
mean_total_col = data['Total'].mean()
data['Total'].fillna(value=mean_total_col, inplace=True)

In [7]:
mode_gender_col = data["Gender"].mode()
data["Gender"].replace("Rather not say", mode_gender_col[0], inplace=True)

In [8]:
gender_outlier = ['Helicopter', 'Computer']
filter_gender = ~data['Gender'].isin(gender_outlier)

data = data[filter_gender]
data.dropna(inplace=True)

In [9]:
def remove_outliers_iqr(data, column_name):
    Q1 = np.percentile(data[column_name], 25)
    Q3 = np.percentile(data[column_name], 75)
    IQR = Q3 - Q1

    maximum = Q3 + (1.5 * IQR)
    minimum = Q1 - (1.5 * IQR)

    outlier_filter = (data[column_name] < minimum) | (
        data[column_name] > maximum)

    data = data[~outlier_filter]

    return data


data = remove_outliers_iqr(data=data, column_name="Unit price")

In [10]:
data = data.drop(["Invoice ID", "Date", "Time"], axis=1)

## Data Preprocessing

### Split

-> bayangkan kalo kita kuliah, yang dipelajari selalu beda kan sama yang keluar di ujian?

In [11]:
from sklearn.model_selection import train_test_split

# memisahkan data menjadi data train dan data test
train, test = train_test_split(data, test_size=0.2, random_state=2023)

print("Banyak data: ", len(data))
print("Banyak data train: ", len(train))
print("Banyak data test: ", len(test))

Banyak data:  1018
Banyak data train:  814
Banyak data test:  204


Kita mau prediksi apa?

In [12]:
data.columns

Index(['Branch', 'City', 'Customer type', 'Gender', 'Product line',
       'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Payment', 'Rating'],
      dtype='object')

In [13]:
label_regresi = "Rating"
label_klasifikasi = "Gender"

-> kita pisahin label dan data train-nya \
-> label == kunci jawaban

In [14]:
# membuang kolom label dari data train
train_data = train.drop([label_regresi, label_klasifikasi], axis=1)

# kolom khusus labelnya aja
train_label_regresi = train[label_regresi]
train_label_klasifikasi = train[label_klasifikasi]

In [15]:
test_data = test.drop([label_regresi, label_klasifikasi], axis=1)
test_label_regresi = test[label_regresi]
test_label_klasifikasi = test[label_klasifikasi]

### Scaling

Misalkan ada statement: \
"cowo filkom lebih sering isi ulang air minum daripada cewe filkom"

-> Apa karena cowo filkom emang sering haus? \
-> Atau karena cowo filkom jumlahnya lebih banyak, makanya waktu observasi sering ketemu cowo?

In [16]:
# Ambil kolom yang bertipe numerik
kolom_numerik = train_data.select_dtypes(include=np.number).columns.tolist()
kolom_numerik

['Unit price', 'Quantity', 'Tax 5%', 'Total']

#### Standard Scaling

- mean jadi 0
- standar deviasi jadi 1

![](https://i.stack.imgur.com/QEPAU.png)

In [17]:
from sklearn.preprocessing import StandardScaler

# membuat objek StandardScaler
scaler = StandardScaler()
# copy data train 
train_ss = train_data.copy()
# fit data train
train_ss[kolom_numerik] = scaler.fit_transform(train[kolom_numerik])
train_ss.head()

,Branch,City,Customer type,Product line,Unit price,Quantity,Tax 5%,Total,Payment
848,C,Naypyitaw,Member,Food and beverages,0.699954,1.222769,1.546624,1.547632,Cash
644,C,Naypyitaw,Member,Electronic accessories,-1.533816,-0.151053,-1.049390,-1.048930,Ewallet
192,C,Naypyitaw,Normal,Food and beverages,1.247839,1.222769,2.131836,2.132967,Cash
682,C,Naypyitaw,Normal,Fashion accessories,-0.165205,0.192403,-0.022307,-0.021630,Ewallet
1057,B,Mandalay,Normal,Fashion accessories,1.691802,-1.181419,-0.441299,-0.440710,Ewallet


In [18]:
# cek standar deviasi dan mean
train_ss.describe()

,Unit price,Quantity,Tax 5%,Total
count,8.140000e+02,8.140000e+02,8.140000e+02,8.140000e+02
mean,2.924224e-16,1.505757e-16,-1.265709e-16,-6.328544e-17
std,1.000615e+00,1.000615e+00,1.000615e+00,1.000615e+00
min,-2.013031e+00,-1.524874e+00,-1.267646e+00,-1.267232e+00
25%,-8.648414e-01,-8.379636e-01,-7.903312e-01,-7.898166e-01
50%,-1.464665e-02,-1.510529e-01,-2.692827e-01,-2.686581e-01
75%,8.292136e-01,8.793133e-01,6.107569e-01,6.115672e-01
max,1.694372e+00,1.566224e+00,3.015667e+00,3.016985e+00


#### Min Max Scaling

- min dan max nya pada rentang tertentu
- biasanya antara 0 dan 1

![](https://michael-fuchs-python.netlify.app/post/2019-08-31-feature-scaling-with-scikit-learn_files/p18p7.png)

In [100]:
from sklearn.preprocessing import MinMaxScaler

# membuat objek MinMaxScaler
minmax = MinMaxScaler()
# copy data train
train_mm = train_data.copy()
# fit data train
train_mm[kolom_numerik] = minmax.fit_transform(train[kolom_numerik])
train_mm.head()

,Branch,City,Customer type,Product line,Unit price,Quantity,Tax 5%,Total,Payment
848,C,Naypyitaw,Member,Food and beverages,0.731775,0.888889,0.657031,0.657031,Cash
644,C,Naypyitaw,Member,Electronic accessories,0.129259,0.444444,0.050955,0.050955,Ewallet
192,C,Naypyitaw,Normal,Food and beverages,0.879556,0.888889,0.793657,0.793657,Cash
682,C,Naypyitaw,Normal,Fashion accessories,0.498415,0.555556,0.290742,0.290742,Ewallet
1057,B,Mandalay,Normal,Fashion accessories,0.999307,0.111111,0.192922,0.192922,Ewallet


In [101]:
# cek min dan max
train_mm.describe()

,count,mean,std,min,25%,50%,75%,max
Unit price,814.0,0.542976,0.269896,0.0,0.309702,0.539025,0.766640,1.0
Quantity,814.0,0.493311,0.323708,0.0,0.222222,0.444444,0.777778,1.0
Tax 5%,814.0,0.295950,0.233608,0.0,0.111436,0.233082,0.438540,1.0
Total,814.0,0.295791,0.233558,0.0,0.111436,0.233082,0.438540,1.0


In [102]:
# misal kita pilih standar scaler aja
test_ss = test_data.copy()
# transform data test
test_ss[kolom_numerik] = scaler.transform(test[kolom_numerik])
test_ss.head()

,Branch,City,Customer type,Product line,Unit price,Quantity,Tax 5%,Total,Payment
158,B,Mandalay,Member,Health and beauty,1.593755,1.222769,2.501319,2.502528,Ewallet
694,C,Naypyitaw,Normal,Food and beverages,-0.044758,1.222769,0.751176,0.752016,Cash
401,C,Naypyitaw,Normal,Home and lifestyle,-0.279042,-0.494508,-0.506235,-0.505660,Credit card
645,A,Yangon,Member,Home and lifestyle,-1.265381,1.222769,-0.552605,-0.552041,Ewallet
885,A,Yangon,Normal,Sports and travel,-0.302543,0.535858,0.078541,0.079239,Cash


### Feature Transformation

In [103]:
train_ss.head()

,Branch,City,Customer type,Product line,Unit price,Quantity,Tax 5%,Total,Payment
848,C,Naypyitaw,Member,Food and beverages,0.699954,1.222769,1.546624,1.547632,Cash
644,C,Naypyitaw,Member,Electronic accessories,-1.533816,-0.151053,-1.049390,-1.048930,Ewallet
192,C,Naypyitaw,Normal,Food and beverages,1.247839,1.222769,2.131836,2.132967,Cash
682,C,Naypyitaw,Normal,Fashion accessories,-0.165205,0.192403,-0.022307,-0.021630,Ewallet
1057,B,Mandalay,Normal,Fashion accessories,1.691802,-1.181419,-0.441299,-0.440710,Ewallet


In [104]:
# Ambil kolom yang bertipe object
kolom_objek = train_ss.select_dtypes(include=np.object).columns.tolist()
kolom_objek

['Branch', 'City', 'Customer type', 'Product line', 'Payment']

![](https://miro.medium.com/v2/resize:fit:1358/1*ggtP4a5YaRx6l09KQaYOnw.png)

In [105]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
# ubah ke bentuk ohe untuk semua kolom objek
train_ohe = ohe.fit_transform(train_ss[kolom_objek]).toarray()
# ubah ke bentuk dataframe
train_ohe = pd.DataFrame(train_ohe, columns=ohe.get_feature_names_out())
# digabung dengan data train 
train_ss = pd.concat([train_ss.reset_index(drop=True), train_ohe], axis=1)
# buang kolom objek karena sudah diubah ke ohe
train_ss = train_ss.drop(kolom_objek, axis=1)

train_ss.head()

,Unit price,Quantity,Tax 5%,Total,Branch_A,Branch_B,Branch_C,City_Mandalay,City_Naypyitaw,City_Yangon,...,Customer type_Normal,Product line_Electronic accessories,Product line_Fashion accessories,Product line_Food and beverages,Product line_Health and beauty,Product line_Home and lifestyle,Product line_Sports and travel,Payment_Cash,Payment_Credit card,Payment_Ewallet
0,0.699954,1.222769,1.546624,1.547632,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.533816,-0.151053,-1.049390,-1.048930,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.247839,1.222769,2.131836,2.132967,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.165205,0.192403,-0.022307,-0.021630,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.691802,-1.181419,-0.441299,-0.440710,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


![](https://miro.medium.com/v2/resize:fit:1400/0*Xhaw5NqAkkqRPxUF.png)

In [106]:
test_ohe = ohe.transform(test_ss[kolom_objek]).toarray()
test_ohe = pd.DataFrame(test_ohe, columns=ohe.get_feature_names_out())
test_ss = test_ss.drop(kolom_objek, axis=1)
test_ss = pd.concat([test_ss.reset_index(drop=True), test_ohe], axis=1)
test_ss.head()

,Unit price,Quantity,Tax 5%,Total,Branch_A,Branch_B,Branch_C,City_Mandalay,City_Naypyitaw,City_Yangon,...,Customer type_Normal,Product line_Electronic accessories,Product line_Fashion accessories,Product line_Food and beverages,Product line_Health and beauty,Product line_Home and lifestyle,Product line_Sports and travel,Payment_Cash,Payment_Credit card,Payment_Ewallet
0,1.593755,1.222769,2.501319,2.502528,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,-0.044758,1.222769,0.751176,0.752016,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.279042,-0.494508,-0.506235,-0.505660,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-1.265381,1.222769,-0.552605,-0.552041,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,-0.302543,0.535858,0.078541,0.079239,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [107]:
# cek info dari data train, udah gaada kolom objek
train_ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814 entries, 0 to 813
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unit price                           814 non-null    float64
 1   Quantity                             814 non-null    float64
 2   Tax 5%                               814 non-null    float64
 3   Total                                814 non-null    float64
 4   Branch_A                             814 non-null    float64
 5   Branch_B                             814 non-null    float64
 6   Branch_C                             814 non-null    float64
 7   City_Mandalay                        814 non-null    float64
 8   City_Naypyitaw                       814 non-null    float64
 9   City_Yangon                          814 non-null    float64
 10  Customer type_Member                 814 non-null    float64
 11  Customer type_Normal            

## Regresi

### Cara Kerja
![](https://miro.medium.com/v2/resize:fit:800/1*nhGPRU12caIw7NK5Rr3p-w.gif) \
source: https://medium.com/swlh/from-animation-to-intuition-linear-regression-and-logistic-regression-f641a31e1caf

### Training

In [108]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

# buat model
model_regresi = LinearRegression()
# model_regresi = Ridge()

# training model
model_regresi.fit(train_ss, train_label_regresi)

LinearRegression()

### Evaluasi

In [109]:
# prediksi
prediksi = model_regresi.predict(test_ss)
prediksi[:5]

array([6.43016893, 7.23670591, 7.0072357 , 7.02245174, 7.16447464])

In [110]:
# lihat kunci jawaban
test_label_regresi[:5]

158    6.0
694    7.6
401    6.2
645    8.7
885    5.0
Name: Rating, dtype: float64

In [111]:
from sklearn.metrics import mean_squared_error

# skor
mse = mean_squared_error(test_label_regresi, prediksi)
rmse = np.sqrt(mse)

print("RMSE: ", rmse)

RMSE:  1.641060290370079


## Klasifikasi

### Cara Kerja
![](https://miro.medium.com/v2/resize:fit:528/0*F1leAlz7TPrVJHY_.gif) \
source: https://saichandra1199.medium.com/classification-vs-regression-622b83ff8e90

In [112]:
train_label_klasifikasi.head()

848     Female
644       Male
192     Female
682     Female
1057    Female
Name: Gender, dtype: object

In [113]:
gender_ke_angka = {
    "Male": 0,
    "Female": 1
}

# ubah ke angka
train_label_klasifikasi = train_label_klasifikasi.replace(gender_ke_angka)
train_label_klasifikasi.head()

848     1
644     0
192     1
682     1
1057    1
Name: Gender, dtype: int64

In [114]:
test_label_klasifikasi = test_label_klasifikasi.replace(gender_ke_angka)
test_label_klasifikasi.head()

158    0
694    1
401    0
645    0
885    0
Name: Gender, dtype: int64

### Training

In [115]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# buat model
model_klasifikasi = LogisticRegression()
# model_klasifikasi = RandomForestClassifier()

# training model
model_klasifikasi.fit(train_ss, train_label_klasifikasi)

LogisticRegression()

### Evaluasi

In [116]:
# prediksi
prediksi = model_klasifikasi.predict(test_ss)
prediksi[:5]

array([0, 1, 0, 0, 1], dtype=int64)

In [117]:
# lihat kunci jawaban
test_label_klasifikasi[:5]

158    0
694    1
401    0
645    0
885    0
Name: Gender, dtype: int64

In [118]:
from sklearn.metrics import accuracy_score

# skor
akurasi = accuracy_score(test_label_klasifikasi, prediksi)
print("Akurasi: ", akurasi)

Akurasi:  0.5441176470588235


# Terima Kasih 🙏🔥🦅

## Tugas 😨